In [ ]:
import datetime
import os
from collections import defaultdict

import dateutil.parser
import pandas as pd
from IPython.display import display
from tqdm.auto import tqdm

from pangaea_downloader.tools import checker

In [ ]:
dirname = "../query-outputs2"

## Load datasets and check distribution of column frequencies

In [ ]:
def find_url_column(df):

    clean_cols = [
        col.lower().replace(" ", "").replace("-", "").replace("_", "").replace(".", "")
        for col in df.columns
    ]
    # Ordered list of priorities
    # Exclude url meta/ref/source which are not links to images
    candidates = [
        "urlimage",
        "urlraw",
        "urlfile",
        "url",
        "urlgraphic",
        "urlthumb",
        "urlthumbnail",
        "image",
        "imagery",
    ]
    for candidate in candidates:
        if candidate not in clean_cols:
            continue
        col = df.columns[clean_cols.index(candidate)]
        if any(df[col].apply(checker.is_url)):
            return col

In [ ]:
column_count = defaultdict(lambda: 0)
column_examples = defaultdict(lambda: [])
files_without_url = []
files_with_repeat_urls = []
n_total = 0
n_valid = 0

for fname in tqdm(os.listdir(dirname)):
    df = pd.read_csv(os.path.join(dirname, fname))
    n_total += 1
    if not checker.has_url_col(df):
        continue
    url_col = find_url_column(df)
    if not url_col:
        print(f"No url column for {fname} with columns\n{df.columns}")
        files_without_url.append(fname)
        continue
    n_valid += 1
    for col in df.columns:
        col = col.lower().strip()
        column_count[col] += 1
        column_examples[col].append(fname)
    subdf = df[df[url_col] != ""]
    if len(subdf) != len(subdf.drop_duplicates(subset=url_col)):
        files_with_repeat_urls.append(fname)

In [ ]:
print(f"There are {n_valid} valid (of {n_total}) valid datasets")
print(
    f"Of which {len(files_with_repeat_urls)} have repeated URLs (possibly multiple annotations)"
)
print()
print(f"There are {len(column_count)} unique column names:")
print()

for col, count in dict(
    sorted(column_count.items(), key=lambda item: item[1], reverse=True)
).items():
    c = col + " "
    print(f"{c:.<35s} {count:4d}")

### Examining columns to find out what their contents are

In [ ]:
df = pd.read_csv(os.path.join(dirname, column_examples["resolution"][0]))
display(df)
print(df.columns)
url_column = find_url_column(df)
print(df[url_column].iloc[0])
print(df["dataset_title"].iloc[0])

In [ ]:
urls = []
n_datasets = 0
for fname in column_examples["seagr cov"]:
    df = pd.read_csv(os.path.join(dirname, fname))
    url_column = find_url_column(df)
    if url_column:
        urls.append(df[url_column])
        n_datasets += 1

In [ ]:
len(pd.concat(urls).unique())

In [ ]:
n_datasets

In [ ]:
len(column_examples["seagr cov"])

In [ ]:
len(set(column_examples["seagr cov"]))

In [ ]:
coverage_columns = [k for k in column_examples.keys() if k.endswith(" cov")]

In [ ]:
coverage_datasets = set()
for k in coverage_columns:
    if k == "ice cov":
        continue
    print()
    print(k)
    print(len(coverage_datasets))
    coverage_datasets.update(column_examples[k])
    print(len(coverage_datasets))

In [ ]:
len(coverage_datasets)

In [ ]:
urls = []
n_datasets = 0
for fname in coverage_datasets:
    df = pd.read_csv(os.path.join(dirname, fname))
    url_column = find_url_column(df)
    if url_column:
        urls.append(df[url_column])
        n_datasets += 1

In [ ]:
len(pd.concat(urls).unique())

In [ ]:
df = pd.read_csv(os.path.join(dirname, column_examples["ice cov"][1]))
display(df)
print(df.columns)
url_column = find_url_column(df)
print(df[url_column].iloc[0])
print(df["dataset_title"].iloc[0])

In [ ]:
df = pd.read_csv(os.path.join(dirname, column_examples["ph"][0]))
display(df)
print(df.columns)
url_column = find_url_column(df)
print(df[url_column].iloc[0])
print(df["dataset_title"].iloc[0])

In [ ]:
df = pd.read_csv(os.path.join(dirname, column_examples["elevation"][-1]))
display(df)
print(df.columns)
url_column = find_url_column(df)
print(df[url_column].iloc[0])
print(df["dataset_title"].iloc[0])

In [ ]:
df = pd.read_csv(os.path.join(dirname, files_with_repeat_urls[3]))
display(df)
print(df.columns)
url_column = find_url_column(df)
print(df[url_column].iloc[0])
print(df["dataset_title"].iloc[-1])

## Implement dataset cleaning functions

In [ ]:
TAXONOMY_RANKS = [
    ["Kingdom", "Regnum"],
    ["Phylum", "Division"],
    ["Ordo", "Order"],
    ["Familia", "Family"],
    ["Genus"],
    ["Species"],
]


def row2taxonomy(row):
    parts = []
    for rank_synonyms in TAXONOMY_RANKS:
        for col in rank_synonyms:
            col_ = col.lower()
            if col in row.keys() and row[col] and row[col] != "-":
                parts.append(row[col])
                break
            elif col_ in row.keys() and row[col_] and row[col_] != "-":
                parts.append(row[col_])
                break
        else:
            break
    return " > ".join(parts)

In [ ]:
def check_title(title):
    """
    Screen dataset title.

    Parameters
    ----------
    title : str
        The title of the dataset.

    Returns
    -------
    bool
        Whether the dataset title is acceptable.
    """

    title = str(title)

    if title.startswith("Meteorological observations"):
        return False
    if title.startswith("Sea ice conditions"):
        return False
    if "topsoil" in title.lower():
        return False
    if "core" in title.lower():
        return False

    return True


def add_file_extension(row):
    """
    Add file extension to image filename.

    Parameters
    ----------
    row : dict
        A dict record which may have fields ``"image"``, ``"File format"``, ``"File type"``.

    Returns
    -------
    fname : str
        File name with extension included.
    """
    if (
        "image" not in row.keys()
        or not row["image"]
        or not isinstance(row["image"], str)
    ):
        return ""

    s = row["image"]
    ext = os.path.splitext(s)[-1]
    if (
        ext.lower()
        in checker.VALID_IMG_EXTENSIONS
        + checker.INVALID_FILE_EXTENSIONS
        + checker.COMPRESSED_FILE_EXTENSIONS
    ):
        return s

    for col in ["File format", "File type"]:
        if col not in row.keys():
            continue
        new_ext = row[col]
        if not new_ext or not isinstance(new_ext, str):
            continue
        new_ext = "." + new_ext.strip().lstrip(".")
        if ext == new_ext:
            break
        s += new_ext
        break

    return s


def datetime2timestamp(ts):
    """
    Convert a datetime string to a timestamp.

    Parameters
    ----------
    ts : str
        Datetime string, in a format understood by ``dateutil.parser``.

    Returns
    -------
    float
        Timestamp; number of seconds since Unix epoch.
    """
    if isinstance(ts, str):
        return dateutil.parser.parse(ts).timestamp()
    return ts


def reformat_df(df, remove_duplicate_columns=True):
    """
    Reformat/clean pangaea dataset.

    Rename columns.

    Parameters
    ----------
    df : pandas.DataFrame
        Original dataset.

    Returns
    -------
    df : pandas.DataFrame or None
        Cleaned dataset, or ``None`` if the dataset is invalid.
    """

    if (
        "dataset_title" in df
        and len(df) > 0
        and df.iloc[0]["dataset_title"]
        and not check_title(df.iloc[0]["dataset_title"])
    ):
        return None

    # Make a copy of the dataframe so we can't overwrite the input
    df = df.copy()

    # Remove bad columns
    df.drop(labels=["-"], axis="columns", inplace=True, errors="ignore")
    # Remove duplicately named columns
    cols_to_drop = []
    if remove_duplicate_columns:
        for col in df.columns:
            if len(col) < 2:
                continue
            if (
                (col[-2] in " _")
                and (col[-1] in "123456789")
                and (col[:-2] in df.columns)
            ):
                cols_to_drop.append(col)
        df.drop(labels=cols_to_drop, axis="columns", inplace=True)

    # Find the correct URL column, and drop other columns containing "url"
    cols_to_drop = []
    mapping = {}
    col_url = find_url_column(df)
    mapping[col_url] = "url"
    for col in df.columns:
        if col != col_url and "url" in col.lower():
            cols_to_drop.append(col)

    # Search for matches to canonical columns.
    # Each entry in desired_columns is a key, value pair where the key
    # is the output column name, and the value is a list of search names
    # in order of priority. The first match will be kept and others discarded.
    desired_columns = {
        "dataset": ["dataset", "Campaign", "campaign"],
        "site": ["Event", "event", "Site", "site", "deployment"],
        "image": ["image", "filename"],
        "timestamp": ["Timestamp"],
        "latitude": ["Latitude", "latitude", "lat", "latitude+"],
        "longitude": ["Longitude", "longitude", "lon", "long", "longitude+"],
        "x_pos": [],
        "y_pos": [],
        "altitude": ["altitude", "height"],
        "depth": [
            "depthwater",
            "bathydepth",
            "bathymetry",
            "bathy",
            "depth",
            "elevation",
        ],
        "backscatter": [],
        "temperature": ["temperature", "temp"],
        "salinity": ["salinity", "sal"],
        "chlorophyll": [],
        "acidity": ["pH"],
        "doi": ["DOI", "doi"],
    }
    # Remove non-alphanumeric padding characters, including spaces, from actual column names
    clean_cols = [
        col.lower().replace(" ", "").replace("-", "").replace("_", "").replace(".", "")
        for col in df.columns
    ]
    # Map to lower case
    lower_cols = [col.lower() for col in clean_cols]

    # Search for matching column names
    for canon, searches in desired_columns.items():
        found = False

        # Check for case-sensitive match
        for search in searches:
            if search not in clean_cols:
                continue
            col = df.columns[clean_cols.index(search)]
            if not found:
                found = True
                mapping[col] = canon
                if col != canon and canon in df.columns:
                    cols_to_drop.append(canon)
            else:
                cols_to_drop.append(col)

        # Check for case-insensitive match
        for search in searches:
            if search.lower() not in lower_cols:
                continue
            col = df.columns[lower_cols.index(search.lower())]
            if not found:
                found = True
                mapping[col] = canon
                if col != canon and canon in df.columns:
                    cols_to_drop.append(canon)
            elif col not in mapping and col not in cols_to_drop:
                cols_to_drop.append(col)

    # Remove superfluous columns
    df.drop(labels=cols_to_drop, axis="columns", inplace=True)
    # Rename columns to canonical names
    df.rename(columns=mapping, inplace=True, errors="raise")

    # Add file extension to image
    df["image"] = df.apply(add_file_extension, axis=1)
    if "timestamp" not in df.columns and "Date/Time" in df.columns:
        df["timestamp"] = df["Date/Time"].apply(datetime2timestamp)

    if any([c in clean_cols for c in ["Kingdom", "Phylum", "Genus"]]):
        df["taxonomy"] = df.apply(row2taxonomy, axis=1)
        df.drop(
            labels=[x for syn in TAXONOMY_RANKS for x in syn],
            axis="columns",
            inplace=True,
            errors="ignore",
        )

    cols_to_drop = [
        "File format",
        "File type",
        "File size",
        "Date/Time",
        "Date/time end",
    ]
    df.drop(labels=cols_to_drop, axis="columns", inplace=True, errors="ignore")

    return df

## Load data with dataset cleaning functions applied

In [ ]:
column_count = defaultdict(lambda: 0)
column_examples = defaultdict(lambda: [])
files_without_url = []
files_with_repeat_urls = []
n_total = 0
n_valid = 0
dfs = []

for fname in tqdm(os.listdir(dirname)):
    df = pd.read_csv(os.path.join(dirname, fname))
    n_total += 1
    if not checker.has_url_col(df):
        continue
    # print(df.columns)
    url_col = find_url_column(df)
    if not url_col:
        # print(f"No url column for {fname} with columns\n{df.columns}")
        files_without_url.append(fname)
        continue
    df = reformat_df(df)
    if df is None:
        continue
    n_valid += 1
    dfs.append(df)
    for col in df.columns:
        column_count[col] += 1
        column_examples[col].append(fname)
    # print(df.columns)
    url_col = "url"
    subdf = df[df[url_col] != ""]
    if len(subdf) != len(subdf.drop_duplicates(subset=url_col)):
        files_with_repeat_urls.append(fname)

In [ ]:
print(f"There are {n_valid} valid (of {n_total}) valid datasets")
print(
    f"Of which {len(files_with_repeat_urls)} have repeated URLs (possibly multiple annotations)"
)
print()
print(f"There are {len(column_count)} unique column names:")
print()

for col, count in dict(
    sorted(column_count.items(), key=lambda item: item[1], reverse=True)
).items():
    c = col + " "
    print(f"{c:.<35s} {count:4d}")

### Merge datasets together

In [ ]:
select_cols = {
    "dataset",
    "site",
    "url",
    "image",
    "timestamp",
    "latitude",
    "longitude",
    "x_pos",
    "y_pos",
    "altitude",
    "depth",
    "backscatter",
    "temperature",
    "salinity",
    "chlorophyll",
    "acidity",
}

df_all = pd.concat([df[df.columns.intersection(select_cols)] for df in dfs])

In [ ]:
df_all

In [ ]:
# Save all records
df_all.to_csv(
    f"../pangaea_{datetime.datetime.today().strftime('%Y-%m-%d')}.csv", index=False
)

In [ ]:
# Filter down to only valid URLs
df_all = df_all[df_all["url"].apply(checker.is_url)]

In [ ]:
len(df_all)

In [ ]:
unique_url_bases = sorted(
    df_all["url"].apply(lambda x: "/".join(x.split("/")[:4])).unique()
)

In [ ]:
len(unique_url_bases)

In [ ]:
url_base = unique_url_bases[0]
df_all[df_all["url"].str.startswith(url_base)].iloc[[0, -1]]

In [ ]:
for i, url_base in enumerate(unique_url_bases):
    print()
    sdf = df_all[df_all["url"].str.startswith(url_base)]
    print(
        "{:3d}/{} ({:7d} URLs), base {}".format(
            i, len(unique_url_bases), len(sdf), url_base
        )
    )
    print(sdf["url"].iloc[0])
    if len(sdf) > 2:
        print(sdf["url"].iloc[1])
    if len(sdf) > 4:
        print(sdf["url"].iloc[len(sdf) // 2])
    if len(sdf) > 3:
        print(sdf["url"].iloc[-2])
    if len(sdf) > 1:
        print(sdf["url"].iloc[-1])

In [ ]:
# Check how many rows have lat & lon
sum(~df_all["latitude"].isna() & ~df_all["longitude"].isna())

In [ ]:
len(df_all["site"].unique())

In [ ]:
# Remove duplicate URLs
df_all = df_all.drop_duplicates(subset="url")

In [ ]:
len(df_all)

In [ ]:
# Check how many rows have lat & lon
sum(~df_all["latitude"].isna() & ~df_all["longitude"].isna())

In [ ]:
len(df_all["site"].unique())

In [ ]:
is_image = df_all["url"].apply(
    lambda x: checker.has_img_extension(x.rstrip("/"))
) | df_all["image"].apply(lambda x: checker.has_img_extension(x.rstrip("/")))
df_all = df_all[is_image]

In [ ]:
len(df_all)

In [ ]:
sum(~df_all["latitude"].isna() & ~df_all["longitude"].isna())

In [ ]:
len(df_all["site"].unique())

In [ ]:
unique_url_bases = sorted(
    df_all["url"].apply(lambda x: "/".join(x.split("/")[:4])).unique()
)

In [ ]:
len(unique_url_bases)

In [ ]:
for i, url_base in enumerate(unique_url_bases):
    print()
    sdf = df_all[df_all["url"].str.startswith(url_base)]
    print(
        "{:3d}/{} ({:7d} URLs), base {}".format(
            i, len(unique_url_bases), len(sdf), url_base
        )
    )
    print(sdf["url"].iloc[0])
    if len(sdf) > 2:
        print(sdf["url"].iloc[1])
    if len(sdf) > 4:
        print(sdf["url"].iloc[len(sdf) // 2])
    if len(sdf) > 12:
        print(sdf["url"].iloc[9])
    if len(sdf) > 102:
        print(sdf["url"].iloc[99])
    if len(sdf) > 1002:
        print(sdf["url"].iloc[999])
    if len(sdf) > 10002:
        print(sdf["url"].iloc[9999])
    if len(sdf) > 3:
        print(sdf["url"].iloc[-2])
    if len(sdf) > 1:
        print(sdf["url"].iloc[-1])

In [ ]:
def check_subdomain(url):
    blacklist = [
        "http://epic.awi.de/",
        "https://epic.awi.de/",
        "http://hdl.handle.net/10013/",
        "http://library.ucsd.edu/dc/object/",
        "https://hs.pangaea.de/Maps/",
        "https://hs.pangaea.de/Movies/",
        "https://hs.pangaea.de/Projects/",
        "https://hs.pangaea.de/bathy/",
        "https://hs.pangaea.de/fishsounder/",
        "https://hs.pangaea.de/mag/",
        "https://hs.pangaea.de/model/",
        "https://hs.pangaea.de/nav/",
        "https://hs.pangaea.de/palaoa/",
        "https://hs.pangaea.de/para/",
        "https://hs.pangaea.de/reflec/",
        "https://hs.pangaea.de/sat/",
        "https://prr.osu.edu/collection/object/",
        "https://store.pangaea.de/Projects/",  # Not all bad, but mostly
        "https://store.pangaea.de/Publications/",  # Not all bad, but mostly
        "https://store.pangaea.de/software/",
        "https://www.ngdc.noaa.gov/geosamples/",
    ]
    for entry in blacklist:
        if url.startswith(entry):
            return False
    return True

In [ ]:
# Remove bad URLs based on their subdomain
df_all = df_all[df_all["url"].apply(check_subdomain)]

In [ ]:
len(df_all)

In [ ]:
unique_url_bases = sorted(
    df_all["url"].apply(lambda x: "/".join(x.split("/")[:5])).unique()
)

In [ ]:
len(unique_url_bases)

In [ ]:
for i, url_base in enumerate(unique_url_bases):
    print()
    sdf = df_all[df_all["url"].str.startswith(url_base)]
    print(
        "{:3d}/{} ({:7d} URLs), base {}".format(
            i, len(unique_url_bases), len(sdf), url_base
        )
    )
    print(sdf["url"].iloc[0])
    if len(sdf) > 2:
        print(sdf["url"].iloc[1])
    if len(sdf) > 4:
        print(sdf["url"].iloc[len(sdf) // 2])
    if len(sdf) > 12:
        print(sdf["url"].iloc[9])
    if len(sdf) > 102:
        print(sdf["url"].iloc[99])
    if len(sdf) > 1002:
        print(sdf["url"].iloc[999])
    if len(sdf) > 10002:
        print(sdf["url"].iloc[9999])
    if len(sdf) > 3:
        print(sdf["url"].iloc[-2])
    if len(sdf) > 1:
        print(sdf["url"].iloc[-1])

In [ ]:
import re

In [ ]:
if re.search(
    "(?<![A-Za-z])map(?![A-Za-z])",
    "https://hs.pangaea.de/Images/ROV/M/M114/GeoB19346-1/data_publish/config/divemap/Dive360_map.jpg",
):
    print("true")

In [ ]:
if re.search(
    "(?<![A-Za-z])map(?![A-Za-z])",
    "https://hs.pangaea.de/Images/ROV/M/M114/GeoB19346-1/data_publish/config/divemap/Dive360mapy.jpg",
):
    print("true")

In [ ]:
def check_subdomain(url):
    blacklist = [
        "http://epic.awi.de/",
        "https://epic.awi.de/",
        "http://hdl.handle.net/10013/",
        "http://library.ucsd.edu/dc/object/",
        "https://app.geosamples.org/uploads/UHM",
        "https://hs.pangaea.de/Images/Linescan",
        "https://hs.pangaea.de/Maps/",
        "https://hs.pangaea.de//Maps",
        "https://hs.pangaea.de/Movies/",
        "https://hs.pangaea.de/Projects/",
        "https://hs.pangaea.de/bathy/",
        "https://hs.pangaea.de/fishsounder/",
        "https://hs.pangaea.de/mag/",
        "https://hs.pangaea.de/model/",
        "https://hs.pangaea.de/nav/",
        "https://hs.pangaea.de/palaoa/",
        "https://hs.pangaea.de/pasata/",
        "https://hs.pangaea.de/para/",
        "https://hs.pangaea.de/polar",
        "https://hs.pangaea.de/reflec/",
        "https://hs.pangaea.de/sat/",
        "https://prr.osu.edu/collection/object/",
        "https://store.pangaea.de/Projects/",  # Not all bad, but mostly
        "https://store.pangaea.de/Publications/",  # Not all bad, but mostly
        "https://store.pangaea.de/software/",
        "https://www.ngdc.noaa.gov/geosamples/",
        "https://hs.pangaea.de/Images/Airphoto",
        # "https://hs.pangaea.de/Images/Cores",  # Some of these are okay
        "https://hs.pangaea.de/Images/Documentation",
        "https://hs.pangaea.de/Images/Maps",
        "https://hs.pangaea.de/Images/Plankton",
        "https://hs.pangaea.de/Images/Satellite",
        "https://hs.pangaea.de/Images/SeaIce",
        "https://hs.pangaea.de/Images/Water",
        "https://store.pangaea.de/Images/Airphoto",
    ]
    banned_words = ["map", "divemap", "dredge_photos", "dredgephotograph"]
    for entry in blacklist:
        if url.startswith(entry):
            return False
    for word in banned_words:
        if re.search("(?<![A-Za-z])" + word + "(?![A-Za-z])", url.lower()):
            return False
    if re.search("(?<![a-z])core(?![a-rty])", url.lower()) and "sur" not in url.lower():
        # Images of cores must contain "surface", or the shorthand "sur"
        return False
    return True

In [ ]:
# Remove bad URLs based on their subdomain (again), and remove maps
df_all = df_all[df_all["url"].apply(check_subdomain)]

In [ ]:
len(df_all)

In [ ]:
unique_url_bases = sorted(
    df_all["url"].apply(lambda x: "/".join(x.split("/")[:5])).unique()
)

In [ ]:
len(unique_url_bases)

In [ ]:
for i, url_base in enumerate(unique_url_bases):
    print()
    sdf = df_all[df_all["url"].str.startswith(url_base)]
    print(
        "{:3d}/{} ({:7d} URLs), base {}".format(
            i, len(unique_url_bases), len(sdf), url_base
        )
    )
    print(sdf["url"].iloc[0])
    if len(sdf) > 2:
        print(sdf["url"].iloc[1])
    if len(sdf) > 4:
        print(sdf["url"].iloc[len(sdf) // 2])
    if len(sdf) > 12:
        print(sdf["url"].iloc[9])
    if len(sdf) > 102:
        print(sdf["url"].iloc[99])
    if len(sdf) > 1002:
        print(sdf["url"].iloc[499])
        print(sdf["url"].iloc[999])
    if len(sdf) > 10002:
        print(sdf["url"].iloc[4999])
        print(sdf["url"].iloc[9999])
    if len(sdf) > 3:
        print(sdf["url"].iloc[-2])
    if len(sdf) > 1:
        print(sdf["url"].iloc[-1])

In [ ]:
df_all.to_csv(
    f"../pangaea_{datetime.datetime.today().strftime('%Y-%m-%d')}_filtered.csv",
    index=False,
)

In [ ]:
cov_dfs = []
for fname in tqdm(coverage_datasets):
    ds_id = os.path.splitext(fname)[0]
    df = pd.read_csv(os.path.join(dirname, fname))
    if not checker.has_url_col(df):
        continue
    # print(df.columns)
    url_col = find_url_column(df)
    if not url_col:
        print(f"No url column for {fname} with columns\n{df.columns}")
        continue
    df["ds_id"] = f"pangaea-{ds_id}"
    df = reformat_df(df, remove_duplicate_columns=False)
    if df is None or len(df) == 0:
        continue
    df = df[~df["url"].isna()]
    df = df[df["url"].apply(check_subdomain)]
    is_image = df["url"].apply(lambda x: checker.has_img_extension(x.rstrip("/"))) | df[
        "image"
    ].apply(lambda x: checker.has_img_extension(x.rstrip("/")))
    df = df[is_image]
    if df is None or len(df) == 0:
        continue
    cov_dfs.append(df)

In [ ]:
df_cov_all = pd.concat(cov_dfs)

In [ ]:
display(df_cov_all)

In [ ]:
for x in sorted(df_cov_all.columns):
    print(x)